# Straight Wire Magnetic Field Demo

This demo showcases the `straight_wire_field` function. It compares the numerically computed magnetic field from a finite straight wire with the known analytical formula.

We compute the Taylor series for the field around a point and compare its coefficients with those from the Taylor series of the analytical formula to verify the function's correctness.

In [1]:
import sys
import numpy as np
from mtflib import MultivariateTaylorFunction, Var, sqrt_taylor
from applications.em.straight_wire import straight_wire_field
import math
import pandas as pd

# --- Global MTF Settings ---
MultivariateTaylorFunction.initialize_mtf(max_order=4, max_dimension=3)
MultivariateTaylorFunction.set_etol(1e-12)

# --- Define Variables for MTF ---
x = Var(1)
y = Var(2)
z = Var(3)

Initializing MTF globals with: _MAX_ORDER=4, _MAX_DIMENSION=3
Loading/Precomputing Taylor coefficients up to order 4
Global precomputed coefficients loading/generation complete.
Size of precomputed_coefficients dictionary in memory: 464 bytes, 0.45 KB, 0.00 MB
MTF globals initialized: _MAX_ORDER=4, _MAX_DIMENSION=3, _INITIALIZED=True
Max coefficient count (order=4, nvars=3): 35
Precomputed coefficients loaded and ready for use.


## 1. Numerical B-Field Calculation

In [2]:
# --- Wire and Field Point Parameters ---
start_point = [0, 0, -1]
end_point = [0, 0, 1]
current = 1.0
num_segments = 200  # More segments for better accuracy

# Define field point for Taylor expansion, e.g., around (0.1, 0, 0)
field_point_mtf = np.array([[x + 0.1, y, z]], dtype=object)

# --- Calculate the B-field numerically ---
B_numerical = straight_wire_field(
    start_point, end_point, current, field_point_mtf, num_segments=num_segments)

B_numerical_vector = B_numerical[0]  # Result has shape (1,3), so get the first row

print("Computed B-field from Straight Wire (Taylor Series Coefficients of By):")
print(B_numerical_vector[1].get_tabular_dataframe())

Computed B-field from Straight Wire (Taylor Series Coefficients of By):
           Coefficient  Order  Exponents
0   1.990074624264e-06      0  (0, 0, 0)
1  -2.009777855384e-05      1  (1, 0, 0)
2   2.000218845409e-04      2  (2, 0, 0)
3  -1.999926239824e-04      2  (0, 2, 0)
4  -2.926055847822e-08      2  (0, 0, 2)
5  -1.999931067866e-03      3  (3, 0, 0)
6   6.000071325056e-03      3  (1, 2, 0)
7  -2.781214576488e-07      3  (1, 0, 2)
8   1.999997060843e-02      4  (4, 0, 0)
9  -1.200000358938e-01      4  (2, 2, 0)
10  2.122432686838e-07      4  (2, 0, 2)
11  1.999999391220e-02      4  (0, 4, 0)
12  7.242063566761e-08      4  (0, 2, 2)
13 -4.744398405518e-08      4  (0, 0, 4)
14  1.542856608563e-01      5  (5, 0, 0)
15  1.282538552870e-01      5  (3, 2, 0)
16 -1.676183975255e-01      5  (3, 0, 2)
17 -2.800000596121e-01      5  (1, 4, 0)
18  1.980959375622e-01      5  (1, 2, 2)
19  3.047574550630e-02      5  (1, 0, 4)
20 -1.854767441064e+00      6  (6, 0, 0)
21 -4.190475888305e-01    

## 2. Analytical B-Field Calculation

In [3]:
# Analytical formula: B = (μ₀ * I / 4π * a) * (cos(θ₁) - cos(θ₂))
# For a wire on the z-axis from z1 to z2, and a point (x,y,z), the field is in the phi direction.
# B_phi = (μ₀ * I / 4π * a) * ( (z - z1)/sqrt((z-z1)²+a²) - (z - z2)/sqrt((z-z2)²+a²) )
# where a = sqrt(x² + y²)

mu_0 = 4 * math.pi * 1e-7
z1 = -1.0
z2 = 1.0

# We are expanding around (0.1, 0, 0), so x_mtf = x + 0.1, y_mtf = y, z_mtf = z
x_mtf = x + 0.1
y_mtf = y
z_mtf = z

a_mtf = sqrt_taylor(x_mtf**2 + y_mtf**2)
cos1_mtf = (z_mtf - z1) / sqrt_taylor((z_mtf - z1)**2 + a_mtf**2)
cos2_mtf = (z_mtf - z2) / sqrt_taylor((z_mtf - z2)**2 + a_mtf**2)

B_phi_mag = (mu_0 * current / (4 * math.pi * a_mtf)) * (cos1_mtf - cos2_mtf)

# The field is in the phi direction. Bx = -B_phi * sin(phi), By = B_phi * cos(phi)
# where phi is the angle in the xy-plane. phi = atan2(y, x)
# cos(phi) = x / a, sin(phi) = y / a
cos_phi_mtf = x_mtf / a_mtf
sin_phi_mtf = y_mtf / a_mtf

Bx_analytical = -B_phi_mag * sin_phi_mtf
By_analytical = B_phi_mag * cos_phi_mtf
Bz_analytical = MultivariateTaylorFunction.from_constant(0.0)

print("Analytical B-field for Straight Wire (Taylor Series Coefficients of By):")
print(By_analytical.get_tabular_dataframe())

Analytical B-field for Straight Wire (Taylor Series Coefficients of By):
            Coefficient  Order   Exponents
0    1.990074380420e-06      0   (0, 0, 0)
1   -2.009778087157e-05      1   (1, 0, 0)
2    2.000218863096e-04      2   (2, 0, 0)
3   -1.999926233788e-04      2   (0, 2, 0)
4   -2.926293079727e-08      2   (0, 0, 2)
5   -1.999931062655e-03      3   (3, 0, 0)
6    6.000071330673e-03      3   (1, 2, 0)
7   -2.781427085682e-07      3   (1, 0, 2)
8    1.999997060351e-02      4   (4, 0, 0)
9   -1.200000358999e-01      4   (2, 2, 0)
10   2.122788823643e-07      4   (2, 0, 2)
11   1.999999391115e-02      4   (0, 4, 0)
12   7.243299702298e-08      4   (0, 2, 2)
13  -4.745197989788e-08      4   (0, 0, 4)
14   1.980151700489e-01      5   (5, 0, 0)
15   9.926217912026e-03      5   (3, 2, 0)
16  -3.537981372738e-06      5   (3, 0, 2)
17  -2.537218029002e-01      5   (1, 4, 0)
18  -3.621649851149e-06      5   (1, 2, 2)
19   2.372598994894e-06      5   (1, 0, 4)
20  -2.019559113963e+00 

## 3. Compare the Results

In [4]:
print("--- Comparison of Taylor Series Coefficients (By component) ---")
df_num = B_numerical_vector[1].get_tabular_dataframe().rename(columns={'Coefficient': 'Numerical'})
df_an = By_analytical.get_tabular_dataframe().rename(columns={'Coefficient': 'Analytical'})

comparison = pd.merge(df_num, df_an, on=['Order', 'Exponents'], how='outer').fillna(0)

comparison['RelativeError'] = (
    np.abs(comparison['Numerical'] - comparison['Analytical']) / 
    np.abs(comparison['Analytical'])
)
comparison['RelativeError'] = (
    comparison['RelativeError'].replace([np.inf, -np.inf], 0).fillna(0)
)

print(comparison[['Exponents', 'Order', 'Numerical', 'Analytical', 'RelativeError']])

--- Comparison of Taylor Series Coefficients (By component) ---
      Exponents  Order           Numerical          Analytical      RelativeError
0     (0, 0, 0)      0  1.990074624264e-06  1.990074380420e-06 1.225300515544e-07
1     (1, 0, 0)      1 -2.009777855384e-05 -2.009778087157e-05 1.153227792694e-07
2     (0, 0, 2)      2 -2.926055847822e-08 -2.926293079727e-08 8.106908596193e-05
3     (0, 2, 0)      2 -1.999926239824e-04 -1.999926233788e-04 3.017792083115e-09
4     (2, 0, 0)      2  2.000218845409e-04  2.000218863096e-04 8.842947621597e-09
5     (1, 0, 2)      3 -2.781214576488e-07 -2.781427085682e-07 7.640293518111e-05
6     (1, 2, 0)      3  6.000071325056e-03  6.000071330673e-03 9.361733047275e-10
7     (3, 0, 0)      3 -1.999931067866e-03 -1.999931062655e-03 2.605725358293e-09
8     (0, 0, 4)      4 -4.744398405518e-08 -4.745197989788e-08 1.685038794128e-04
9     (0, 2, 2)      4  7.242063566761e-08  7.243299702298e-08 1.706591730219e-04
10    (0, 4, 0)      4  1.99999939